In [ ]:
# Collaborative filtering
# Constest based filtering
# Movie dataset from Kaggle
# Preprocessing
# cleaning the data
# Understand how to do collaborative filtering
# By using similarity score 
# 

In [1]:
#Goals

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Loading dataset
movies_df = pd.read_csv("./ml-latest/movies.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies_df[movies_df["title"] == "Gladiator (2000)"]

,movieId,title,genres
3488,3578,Gladiator (2000),Action|Adventure|Drama


In [14]:
print(movies_df.size)
print(movies_df.shape)
print(movies_df.dtypes)
print(movies_df.info())
print(movies_df.describe())

102624
(34208, 3)
movieId     int64
title      object
genres     object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34208 entries, 0 to 34207
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  34208 non-null  int64 
 1   title    34208 non-null  object
 2   genres   34208 non-null  object
dtypes: int64(1), object(2)
memory usage: 801.9+ KB
None
             movieId
count   34208.000000
mean    75585.571445
std     50726.054238
min         1.000000
25%     26008.750000
50%     86452.000000
75%    119454.500000
max    151711.000000


In [ ]:
# Preprocessing
